In [12]:
import torch
from byol_pytorch import BYOL
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import importlib
import Features.backbone.Custom as Custom
from IPython.display import display,clear_output
import torchvision as tv
import pickle
import kornia.augmentation as K
import Features.backbone.MiraBest as mb
importlib.reload(Custom)
import random



# Define the directory of your dataset
meerkatn_dir = "/idia/projects/hippo/Koketso/meerkat"
meerkat_dir = "/idia/projects/hippo/Koketso/meerkat_resized"
meerkat_curated = "/idia/projects/hippo/Koketso/meerkat/meerkat_curated"



#Galaxy_zoo directories
galaxyzooq_dir = "/idia/projects/hippo/Koketso/galaxyzoo/resized/galaxy_zoo_class"
galaxyzooqn_dir = "/idia/projects/hippo/Koketso/galaxyzoo/nonresized/galaxy_zoo_class"
galaxyzooqns_dir = "/idia/projects/hippo/Koketso/galaxyzoo/nonresized/galaxy_zoo_class_sigma"



galaxyzoo_dir = "/idia/projects/hippo/Koketso/galaxyzoo/galaxy_zoo"

#Hand_sign dataset

hand_dir = "/idia/projects/hippo/Koketso/Train_Alphabet"
handt_dir = "/idia/projects/hippo/Koketso/Test_Alphabet"

#Imagenet
im13_dir = "/idia/projects/hippo/Koketso/ILSVRC/Data/DET/train/ILSVRC2013_train"
im12_dir =  "/idia/projects/hippo/Koketso/ILSVRC/Data/DET/test"
dogbreed_dir = "/idia/projects/hippo/Koketso/dog_breeds"

In [13]:
#Zoobot
"""

model = tv.models.efficientnet_b0(weights = "IMAGENET1K_V1")
#zoobot_dict = torch.load("Features/models_/old_models/checkpoints/epoch=17-step=2808.ckpt")['state_dict']
#loaded_dict = {k: zoobot_dict["model.0.features." +k] for k in model.features.state_dict()}

#model.classifier[1] = torch.nn.Linear(1280,2)
#model.load_state_dict(torch.load("Features/models_/mirabest_efficientNet_sup.pt",map_location = torch.device("cpu"))['model_state_dict'])
model.classifier[1] = torch.nn.Linear(1280,100)
model.classifier[1].weight.data.normal_(0,0.01)

"""



#Resnet

model = tv.models.resnet18(weights = "IMAGENET1K_V1")

#model = torchvision.models.resnet18(weights = "IMAGENET1K_V1")

model.fc = torch.nn.Linear(512,100)
model.fc.weight.data.normal_(0,0.01)
model.load_state_dict(torch.load("Features/models_/res18byol_0.pt")['model_state_dict'])


#model.load_state_dict(torch.load("Featuremodels_/model_handhfzb_resnet18.pt",map_location = torch.device("cpu")))
#model.load_state_dict(torch.load("Features/models_/r34trigzbyol.pt",map_location = torch.device("cpu"))['model_state_dict'])

#model.load_state_dict(torch.load("Features/models/model_gz_99HVcut.pt",map_location = torch.device("cpu")))

model.train()

#model.load_state_dict(torch.load("Features/models/modelbyogzresized3.pt"))

#model.load_state_dict(torch.load("Features/models/gzcbi_rot_resized_byolstzooms.pt"))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
torch.load("Features/models_/res18byol_0.pt")['epoch']

8

In [14]:
#resnet.load_state_dict(torch.load("./Features/models/gzcbirotate_improved-net.pt"))

# Here I write an augmentation that includes all rotation angles. and cuts out the edges
class RandomRotationWithCrop(K.RandomRotation):
    def __init__(self, degrees, crop_size, output_dim = 244,p = 0.5):
        super(RandomRotationWithCrop, self).__init__(degrees, p = 1)
        #super(RandomRotationWithCrop, self).__init__(crop_size)
        self.rotation_transform = K.RandomRotation(degrees)
        self.crop_transform = K.CenterCrop(crop_size, keepdim = False,align_corners = True)
        self.resize_transform = K.Resize(output_dim)
    def __call__(self, x):
        if random.random() <self.p:
            # Apply random rotation
            x = self.rotation_transform(x)

            # Apply center crop
            x = self.crop_transform(x)
            x = self.resize_transform(x)

        return x


augment_fn = torch.nn.Sequential(
    
   
        #rotation by an angle of 90
        K.RandomRotation([90,90], p=0.15),
        K.RandomRotation([180,180], p=0.15),
        K.RandomRotation([270,270], p=0.15),
        RandomRotationWithCrop(degrees = [0,360],crop_size =210,p =0.3),
    
        kornia.augmentation.RandomResizedCrop([200,200], p = 0.2),
        kornia.augmentation.RandomGaussianBlur(kernel_size = [3,3],sigma = [1,2], p =0.1)
    
)



learner = BYOL(
    model,
    image_size = 244,
    hidden_layer = 'avgpool',
    augment_fn = augment_fn
   
)

In [15]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
learner = learner.to(device)

In [16]:
batch_size = 100

resize = 250

dir_ = galaxyzoo_dir
#dir_ = meerkat_curated

dataset = Custom.dataset(dir_)
names = [name[0].split('/')[-1] for name in dataset.imgs]
importlib.reload(Custom)

datasets = Custom.train_val_dataset(dataset, val_split = 0.05)

#Traning

transformed_train_dataset = Custom.Custom(datasets['train'],
                                    names = names,
                                    resize = resize,
                                   crop = 224,
                                   )

transformed_val_dataset = Custom.Custom(datasets['val'],
                                    names = names,
                                    resize = resize,
                                   crop = 224,
                                   )

#Validation

In [19]:
loader = DataLoader(transformed_train_dataset, 
                    batch_size, 
                    shuffle = True,
                    num_workers = 16)

val_loader = DataLoader(transformed_val_dataset, 
                    batch_size, 
                    shuffle = True,
                    num_workers = 16)

patience = 5

best_loss = 5000000

opt = torch.optim.Adam(learner.parameters(), lr=3e-5)    #3e-7 for gz   -4 for hand  -5 for meerkat_curated

In [7]:
#Mirabest
transform = tv.transforms.Compose([
                            #tv.transforms.ToPILImage(),
                            #tv.transforms.Resize((424,424)),
                            tv.transforms.Resize((300,300)),
                            tv.transforms.CenterCrop(224),           # So they are compatible with the dnn models
                            tv.transforms.Grayscale(num_output_channels=3),
                            tv.transforms.ToTensor(),

                            #tv.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                            ])

#transformed_dataset = mb.MBFRConfident(root='./batches', train=True, download=True, transform=transform) 

In [ ]:
loss_saved = [[],[]]
loss_pickle = "./Features/losses/res18byol_1.plk"


with open(loss_pickle,'wb') as file:
    pickle.dump(loss_saved,file)
for epoch in range(300):
    loss_ = 0.0
    for i,Images in enumerate(loader):
        Images = Images[0]
        #send imaged to device
        images = Images.to(device)
        #optain loss
        loss = learner(images)
        
        #optimization steps
        opt.zero_grad()
        loss.backward()
        opt.step()
        learner.update_moving_average() #update moving average of target encoder
        loss_ += loss.item()
        #display(progress)
        if i%5 ==0:
            print("Batch epoch :"+ str(epoch) + " Loss :" + str(loss.item()))
        try:
            if i%500 ==0:
                clear_output(wait = True)
   
        except:
            print("didnt work")
    loss_saved[0].append(loss_)
    
    
    #Implementing the early stopping
    # Validate
    clear_output(wait = True)
    print("Validating")
    with torch.no_grad():
        val_loss = 0
        for i, val_images in enumerate(val_loader):
            
            val_images = val_images[0]
            val_images = val_images.to(device)
            v_loss = learner(val_images)
            val_loss += v_loss.item() 
            
            loss_saved[1].append(val_loss)
        print("Validation loss: ",val_loss)

    if val_loss < best_loss:
        
        best_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            }, "./Features/models_/res18byol_1.pt")
        loss_saved[1].append(loss_)
    
        counter = 0
    else:
        counter += 1

    # Check if early stopping criteria are met
    if counter >= patience:
        print("Early stopping: No improvement in validation loss for {} epochs".format(patience))
        break
        
    #save model and  loss
    #torch.save(model.state_dict(), "./Features/models_/model_gzrcr_res34.pt")

    

Batch epoch :2 Loss :0.17953573167324066
Batch epoch :2 Loss :0.22150404751300812
Batch epoch :2 Loss :0.35205909609794617
Batch epoch :2 Loss :0.2068321406841278
Batch epoch :2 Loss :0.054528430104255676
Batch epoch :2 Loss :0.24212221801280975
Batch epoch :2 Loss :0.07840163260698318
Batch epoch :2 Loss :0.08725753426551819
Batch epoch :2 Loss :0.278033047914505
Batch epoch :2 Loss :0.26980650424957275
Batch epoch :2 Loss :0.05299478396773338
Batch epoch :2 Loss :0.06412839889526367
Batch epoch :2 Loss :0.07730522751808167
Batch epoch :2 Loss :0.056511957198381424
Batch epoch :2 Loss :0.4167291820049286
Batch epoch :2 Loss :0.20618194341659546
Batch epoch :2 Loss :0.07312589138746262
Batch epoch :2 Loss :0.20156630873680115
Batch epoch :2 Loss :0.25471508502960205
Batch epoch :2 Loss :0.07100146263837814
Batch epoch :2 Loss :0.25665727257728577
Batch epoch :2 Loss :0.3061891794204712
Batch epoch :2 Loss :0.34192773699760437
Batch epoch :2 Loss :0.06397383660078049
Batch epoch :2 Loss

In [49]:
with torch.no_grad():
    val_loss = 0
    for i, val_images in enumerate(val_loader):
        val_images = val_images[0]
        val_images = val_images.to(device)
        v_loss = learner(val_images)
        val_loss += v_loss.item() 
            
        loss_saved[1].append(val_loss)
        
        

In [ ]:
import pickle
with open('ems.pkl','wb') as file:
    pickle.dump(embeddings,file)

In [ ]:
torch.save(model.state_dict(), "./Features/models/gzcbi_rot_resized_byol.pt")        

# save your improved network